# CNN for face-anti-spoofing
We all know that, the face anti-spoofing is an technique that could prevent possible face-spoofing attack. For example, an imposter might use a photo of the legal user to "try to pretend a real user" and foul the face recognition system. Thus it is important to use the face anti-spoofint technique to enhance the security of the system. to be able to to so, we are going to train a CNN again.

### MOUNTING GOOGLE DRIVE
The first thing to do is to give google colab permission to access our drive so as to save the training checkpoints.

In [ ]:
#MOUNTING GOOGLE DRIVE

from google.colab import drive
drive.mount('/content/gdrive',  force_remount=True) 

# force_remount is an argument to force google drive to mount once again.

### IMPORT LIBRARIES


In [4]:
from keras.layers import Dense, Dropout, Input, Flatten
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.models import model_from_json
import json
import os

### DOWNLOAD The DATASET

In [ ]:
%%capture
# Download dataset from google drive

!gdown --id 1C7xUQkgh4hyZOQVgr_5FLT_66Ok12Uwe

# unzip the archive file
!unzip fake_or_real.zip

# we don't need the archive file anymore
!rm fake_or_real.zip

## Configuring directories
We are going to introduce the directories and separate the spoof and real identities


In [6]:
train_dir = os.path.join('/content', 'fake_or_real')

label_name = [subdir for subdir in sorted(os.listdir(train_dir))]

print(f"Classes\t: {label_name}")

Classes	: ['real', 'spoof']


## LOAD AND GENERATION

Now we have everything to start the training phase,
Let's define some constants for our deep architecture. Each of the constants has its own application which is summarized in the table below:

CONSTANT_NAME | APPLICATION
-------------------|------------------
BATCH_SIZE       | # samples that will be passed through to the network at one time
IMAGE_SIZE       | width of the image 
IMAGE_SHAPE      | shape of the image (image_height, image_width)
N_CLASSES        | the output of the model, the classification layer of the model


In [12]:
BATCH_SIZE = 25
IMAGE_SIZE = 160
IMAGE_SHAPE = (IMAGE_SIZE, IMAGE_SIZE) 
N_CLASSES = 2

### DATA GENERATOR
We all encountered a situation where we try to load a dataset but there is not enough memory in our machine. 

As the field of machine learning progresses, this problem becomes more and more common. This is already one of the challenges in the field of vision where large datasets of images and video files are processed.

Here, we will use `Keras` to build data generators for loading and processing our images

The `ImageDataGenerator` class is very useful in image classification. There are several ways to use this generator, depending on the method we use, here we will focus on `flow_from_directory` which takes a path to the directory containing images sorted in sub directories and image augmentation parameters.

In [11]:
from tensorflow import keras

train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30, # rotation
    width_shift_range=0.2,
    height_shift_range=0.2,
    brightness_range=(0.8, 1.2), # illumination
    fill_mode='nearest',
    shear_range=0.2,
    zoom_range=0.3,
    rescale=1./255,
    validation_split=0.2,
)

In [14]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SHAPE,
    color_mode='rgb',
    class_mode='binary',
    batch_size=BATCH_SIZE,
    shuffle=True,
    subset="training"
)

valid_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMAGE_SHAPE,
    color_mode='rgb',
    class_mode='binary',
    batch_size=BATCH_SIZE, 
    subset="validation"
)

Found 10092 images belonging to 2 classes.
Found 2522 images belonging to 2 classes.


As you can see we have **10092** images which correspond to 2 different classes (real or spoof) in our dataset where will be used as the training samples.

What we want to achive here is to train a model to be able to extract our features from the images.

## Training with MobileNetV2

### Why MobileNet?
MobileNet is an architecture which is more suitable for mobile and embedded based vision applications where there is lack of compute power. This architecture was proposed by Google.

As the system already computationaly expensive duo to emossion, gesture, face, and etc detection, we decided to decrease the pressure and train a simplare architecture.

In [16]:
# We used imagenet pre-trained weights to initialized the net

mobilenet = MobileNetV2(
    weights = "imagenet",
    include_top = False,
    input_tensor=Input(shape=(IMAGE_SIZE,IMAGE_SIZE,3))
)

9420800/9406464 [==============================] - 0s 0us/step


In [17]:
mobilenet.trainable = False

In [19]:
output = Flatten()(mobilenet.output)
output = Dropout(0.3)(output)
output = Dense(units = 8,activation='relu')(output)
prediction = Dense(1,activation='sigmoid')(output) # (probability predictions (binary))

In [20]:
model = Model(inputs = mobilenet.input,outputs = prediction)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 160, 160, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv1 (Conv2D)                 (None, 80, 80, 32)   864         ['input_1[0][0]']                
                                                                                                  
 bn_Conv1 (BatchNormalization)  (None, 80, 80, 32)   128         ['Conv1[0][0]']                  
                                                                                                  
 Conv1_relu (ReLU)              (None, 80, 80, 32)   0           ['bn_Conv1[0][0]']           

## Callback for TRAINING
---
for our model we are going to take the most use out of some callback function in the keras api.
- <b>`ModelCheckpoint`</b> 🏁: callback is used in conjunction with training using `model.fit()` to save a model or weights (in a checkpoint file) at some interval, so the model or weights can be loaded later to continue the training from the state saved.

- <b>`EarlyStopping` </b>🚦: Assuming the goal of a training is to minimize the loss. With this, the metric to be monitored would be `loss`, and mode would be `min`. A `model.fit()` training loop will check at end of every epoch whether the `loss` is no longer decreasing, considering the `min_delta` and patience if applicable. Once it's found no longer decreasing,the training terminates.


In [21]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

In [27]:
# stop training if no improvements are seen
early_stop = EarlyStopping(monitor="val_loss",
                            mode="min",
                            patience=3,
                            restore_best_weights=True)

# saves model weights to file
checkpoint = ModelCheckpoint(
    os.path.join(
        '/content/gdrive/MyDrive/Multimodal_Interaction/fake_or_real/model', 'antispoofing_model_{epoch:02d}-{val_accuracy:.6f}.h5'
        ),
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
    mode='min',
    save_weights_only=True)


In [28]:
model.compile(
  loss='binary_crossentropy',
  optimizer=Adam(
    learning_rate=0.000001,
    beta_1=0.9,
    beta_2=0.999,
    epsilon=1e-07
),
  metrics=['accuracy']
)

In [ ]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // 25,
    validation_data = valid_generator, 
    validation_steps = valid_generator.samples // 25,
    epochs = 100,
    callbacks=[checkpoint, early_stop])

## Save model architecture
We save the best model and its architecture in order to use it in real time

In [ ]:
model.save('/content/gdrive/MyDrive/Multimodal_Interaction/fake_or_real/model/fake_or_real.h5')

In [ ]:
# serialize model to JSON

model_json = model.to_json()
with open("/content/gdrive/MyDrive/Multimodal_Interaction/fake_or_real/model/model_net.json", "w") as json_file:
    json_file.write(model_json)